## Задание

Добейтесь на автокодировщике с 2-мерным скрытым пространством на 3-х цифрах: 0, 1 и 3 – ошибки MSE**<0.034** на скорости обучения **0.001** на **10-й эпохе**.

## Импорт библиотек


In [ ]:
# Работа с операционной системой
import os

# Отрисовка графиков
import matplotlib.pyplot as plt

# Операции с путями
import glob

# Работа с массивами данных
import numpy as np

# Слои
from tensorflow.keras.layers import Dense, Flatten, Reshape, Input, Conv2DTranspose, concatenate, Activation, MaxPooling2D, Conv2D, BatchNormalization, Concatenate

# Модель
from tensorflow.keras import Model

# Загрузка модели
from tensorflow.keras.models import load_model

# Датасет
from tensorflow.keras.datasets import mnist

# Оптимизатор для обучения модели
from tensorflow.keras.optimizers import Adam

# Коллбэки для выдачи информации в процессе обучения
from tensorflow.keras.callbacks import LambdaCallback

%matplotlib inline

## Утилиты

Удаление изображений. Применять при обучении новой модели, чтобы не было путаницы в картинках.

In [ ]:
def clean():
  # Получение названий всех картинок
  paths = glob.glob('*.jpg')

  # Удаление всех картинок по полученным путям
  for p in paths:
    os.remove(p)

# Удаление всех картинок
clean()

## Загрузка данных

In [ ]:
# Загрузка датасета
(X_train, y_train), (X_test, y_test) = mnist.load_data()

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
# Загрузка и подготовка данных
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = (X_train.astype('float32') / 255.).reshape(-1, 28*28)
X_test = (X_test.astype('float32') / 255.).reshape(-1, 28*28)

In [ ]:
# Выбор визуализируемых классов (цифр) и формирование подвыборок для них по маске
numbers = [0, 1, 3]
mask = np.array([(i in numbers) for i in y_train])
X_train = X_train[mask]
y_train = y_train[mask]

## Создание модели и обучение

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Reshape, Conv2DTranspose, UpSampling2D, LeakyReLU, Dropout, GaussianNoise
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import mnist
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.regularizers import l2

Функция-коллбэк. Отрисовывает объекты в скрытом пространстве

In [ ]:
def ae_on_epoch_end(epoch, logs):
    print('________________________')
    print(f'*** ЭПОХА: {epoch+1}, loss: {logs["loss"]} ***')
    print('________________________')

    # Получение картинки латентного пространства в конце эпохи и запись в файл
    # Задание числа пикселей на дюйм
    plt.figure(dpi=100)

    # Предсказание енкодера на тренировочной выборке
    predict = encoder.predict(X_train)

    # Создание рисунка: множество точек на плоскости 3-х цветов (3-х классов)
    scatter = plt.scatter(predict[:,0,],predict[:,1], c=y_train, alpha=0.6, s=5)

    # Создание легенды
    legend2 = plt.legend(*scatter.legend_elements(), loc='upper right', title='Классы')

    # Сохранение картинки с названием, которого еще нет
    paths = glob.glob('*.jpg')
    plt.savefig(f'image_{str(len(paths))}.jpg')

    # Отображение. Без него рисунок не отрисуется
    plt.show()

ae_callback = LambdaCallback(on_epoch_end=ae_on_epoch_end)

In [ ]:
# Архитектура автокодировщика
input_img = Input(shape=(28, 28, 1))

In [ ]:
# Архитектура (почти как у вас, с 3 ключевыми изменениями)

encoded = Dense(128, activation='relu', kernel_initializer='he_normal')(input_img)  # 1. Инициализация He
encoded = Dense(64, activation='relu', kernel_initializer='he_normal')(encoded)     # 2. Инициализация He
encoded = Dense(2, activation='linear')(encoded)

decoded = Dense(64, activation='relu')(encoded)
decoded = Dense(128, activation='relu')(decoded)
decoded = Dense(784, activation='sigmoid')(decoded)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer=Adam(learning_rate=0.001, clipvalue=0.5), loss='mse')  # 3. Добавлен clipvalue
encoder = Model(input_img, encoded)  # Отдельная модель для энкодера

In [ ]:
# Обучение
history = autoencoder.fit(
    X_train, X_train,
    epochs=10,
    batch_size=128,
    validation_data=(X_test, X_test),
    verbose=1,
     callbacks=[ae_callback]
)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# Результаты
print(f"Final Train MSE: {history.history['loss'][-1]:.5f}")
print(f"Final Val MSE: {history.history['val_loss'][-1]:.5f}")

Final Train MSE: 0.02812
Final Val MSE: 0.06063
